<a href="https://colab.research.google.com/github/emreaskinsoftware/Kalp-Hastaligi-Tahmini/blob/main/Kalp_Hastaligi_Tahmini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

print("Lütfen 'kaggle.json' dosyanızı seçin:")
files.upload()

Lütfen 'kaggle.json' dosyanızı seçin:


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"emreaskin","key":"2e2b160eb3d4114495adcbc0719f77f3"}'}

In [ ]:
from google.colab import files

print("Lütfen bilgisayarınızdaki 'heart.csv' dosyasını seçin:")
# (Az önce ZIP'ten çıkardığınız dosyayı seçeceksiniz)
uploaded = files.upload()

Lütfen bilgisayarınızdaki 'heart.csv' dosyasını seçin:


Saving heart.csv to heart.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Seaborn temasını ayarlayalım
sns.set_theme(style="whitegrid")

# Veri setini yüklüyoruz
try:
    df = pd.read_csv('heart.csv')
    print("✓ 'heart.csv' başarıyla DataFrame'e yüklendi.")
    print(f"Veri setinin boyutu (Hasta Sayısı, Özellik Sayısı): {df.shape}")
except FileNotFoundError:
    print("Hata: 'heart.csv' dosyası bulunamadı.")

# --- İLK ANALİZ (EDA) ---

# Verinin yapısını anlamak için ilk 5 satıra bakalım
print("\n--- Veri Setinden İlk 5 Satır (head) ---")
print(df.head())

# Veri setinin genel özetine (sütun tipleri, boş değerler) bakalım
print("\n--- Veri Seti Bilgileri (info) ---")
df.info()

# Sayısal sütunların istatistiksel özetine bakalım
print("\n--- Sayısal Veri Özeti (describe) ---")
print(df.describe())

✓ 'heart.csv' başarıyla DataFrame'e yüklendi.
Veri setinin boyutu (Hasta Sayısı, Özellik Sayısı): (920, 16)

--- Veri Setinden İlk 5 Satır (head) ---
   id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num 

In [ ]:
# --- Adım 2.1: En Temiz Alt Kümeyi (Cleveland) Seçme ---

# 'dataset' sütunundaki benzersiz değerlere ve sayılarına bakalım
print("--- Orijinal Veri Setindeki Kaynaklar ---")
print(df['dataset'].value_counts())

# Sadece 'Cleveland' verisini seçip yeni bir DataFrame'e kopyalıyoruz
# .copy() kullanmak, Pandas'taki 'SettingWithCopyWarning' uyarısını engeller
df_clean = df[df['dataset'] == 'Cleveland'].copy()

print(f"\n✓ Filtreleme tamamlandı. Yeni veri seti boyutu: {df_clean.shape}")

# Şimdi BU YENİ DataFrame'in .info() çıktısına bakalım
print("\n--- 'df_clean' (Cleveland) Veri Seti Bilgileri ---")
df_clean.info()

--- Orijinal Veri Setindeki Kaynaklar ---
dataset
Cleveland        304
Hungary          293
VA Long Beach    200
Switzerland      123
Name: count, dtype: int64

✓ Filtreleme tamamlandı. Yeni veri seti boyutu: (304, 16)

--- 'df_clean' (Cleveland) Veri Seti Bilgileri ---
<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 303
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        304 non-null    int64  
 1   age       304 non-null    int64  
 2   sex       304 non-null    object 
 3   dataset   304 non-null    object 
 4   cp        304 non-null    object 
 5   trestbps  304 non-null    float64
 6   chol      304 non-null    float64
 7   fbs       304 non-null    object 
 8   restecg   304 non-null    object 
 9   thalch    304 non-null    float64
 10  exang     304 non-null    object 
 11  oldpeak   304 non-null    float64
 12  slope     303 non-null    object 
 13  ca        299 non-null    float64
 14  

In [ ]:
# --- Adım 2.2: Son Temizlik ve ML Ön İşleme (Preprocessing) ---

# 1. (Gereksiz) 'id' ve 'dataset' sütunlarını silerek başlayalım
df_clean = df_clean.drop(['id', 'dataset'], axis=1)

# 2. (Kararımız) Az sayıdaki eksik veriyi içeren SATIRLARI sil
# 304 satırdan geriye kaç satır kaldığına bakalım
df_clean = df_clean.dropna()

print(f"Eksik veriler silindi. Kalan satır sayısı: {len(df_clean)}")

# 3. (ML Gereksinimi) Hedef Değişkenimizi (num) hazırlayalım
# 'num' sütunundaki değer 0'dan büyükse (1,2,3,4) -> 1 (Hasta)
# 0 ise -> 0 (Sağlıklı) yap.
df_clean['target'] = (df_clean['num'] > 0).astype(int)

# Orijinal 'num' sütununa artık ihtiyacımız kalmadı, onu silelim
df_clean = df_clean.drop('num', axis=1)

# 4. (ML Gereksinimi) Kategorik (object) sütunları sayısala çevirelim
# pd.get_dummies() fonksiyonu 'object' ve 'bool' tipindeki tüm sütunları
# otomatik olarak bulup One-Hot Encoding uygular.
df_final = pd.get_dummies(df_clean, drop_first=True)

print("✓ Kategorik sütunlar sayısala dönüştürüldü.")

# --- SON KONTROL ---
print("\n--- Makine Öğrenimine Hazır 'df_final' Verisi ---")
df_final.info()

print("\n--- 'df_final' Verisinden İlk 5 Satır ---")
print(df_final.head())

Eksik veriler silindi. Kalan satır sayısı: 297
✓ Kategorik sütunlar sayısala dönüştürüldü.

--- Makine Öğrenimine Hazır 'df_final' Verisi ---
<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 0 to 301
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       297 non-null    int64  
 1   trestbps                  297 non-null    float64
 2   chol                      297 non-null    float64
 3   thalch                    297 non-null    float64
 4   oldpeak                   297 non-null    float64
 5   ca                        297 non-null    float64
 6   target                    297 non-null    int64  
 7   sex_Male                  297 non-null    bool   
 8   cp_atypical angina        297 non-null    bool   
 9   cp_non-anginal            297 non-null    bool   
 10  cp_typical angina         297 non-null    bool   
 11  fbs_True                  297 non-null

In [ ]:
# --- Adım 3.1: Özellikler (X) ve Hedef (y) Ayırma ---

# X = 'target' hariç TÜM sütunlar
X = df_final.drop('target', axis=1)

# y = SADECE 'target' sütunu
y = df_final['target']

print("--- Özellikler (X) ---")
print(X.head())
print("\n--- Hedef (y) ---")
print(y.head())

--- Özellikler (X) ---
   age  trestbps   chol  thalch  oldpeak   ca  sex_Male  cp_atypical angina  \
0   63     145.0  233.0   150.0      2.3  0.0      True               False   
1   67     160.0  286.0   108.0      1.5  3.0      True               False   
2   67     120.0  229.0   129.0      2.6  2.0      True               False   
3   37     130.0  250.0   187.0      3.5  0.0      True               False   
4   41     130.0  204.0   172.0      1.4  0.0     False                True   

   cp_non-anginal  cp_typical angina  fbs_True  restecg_normal  \
0           False               True      True           False   
1           False              False     False           False   
2           False              False     False           False   
3            True              False     False            True   
4           False              False     False           False   

   restecg_st-t abnormality  exang_True  slope_flat  slope_upsloping  \
0                     False      

In [ ]:
# Gerekli fonksiyonu sklearn'den 'import' edelim
from sklearn.model_selection import train_test_split

# Verimizi ayıralım
# test_size=0.2 -> Verinin %20'sini test (sınav) için ayır, %80'i eğitim (çalışma)
# random_state=42 -> Bu, projemizi her çalıştırdığımızda aynı 'rastgele'
#                    bölünmeyi yapmasını sağlar, böylece sonuçlarımız
#                    tekrarlanabilir olur. (42, sektörde bir gelenektir)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Toplam veri: {len(df_final)} satır")
print(f"Eğitim seti boyutu (X_train): {len(X_train)} satır (%80)")
print(f"Test seti boyutu (X_test): {len(X_test)} satır (%20)")

Toplam veri: 297 satır
Eğitim seti boyutu (X_train): 237 satır (%80)
Test seti boyutu (X_test): 60 satır (%20)


In [ ]:
# Gerekli model ve metrikleri 'import' edelim
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. Modeli oluştur
# max_iter=1000 -> Modelin en iyi çözümü bulmak için deneme sayısını
#                  artırır, bu olası bir 'uyarı'yı (warning) engeller.
model = LogisticRegression(max_iter=1000)

# 2. Modeli EĞİT (Fit)
# Model, X_train (sorular) ve y_train (cevaplar) arasındaki ilişkiyi öğrenir
model.fit(X_train, y_train)

print("✓ Model başarıyla eğitildi!")

✓ Model başarıyla eğitildi!


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 3. TAHMİN Yap (Predict)
# Modelden X_test'teki hastalar için tahmin yapmasını istiyoruz
y_pred = model.predict(X_test)

# 4. DEĞERLENDİR (Evaluate)
# Modelin tahminlerini (y_pred) gerçek cevaplarla (y_test) karşılaştır

# a) Doğruluk (Accuracy) Skoru
accuracy = accuracy_score(y_test, y_pred)
print(f"--- Model Doğruluk (Accuracy) Skoru ---")
print(f"{accuracy * 100:.2f}%")
print(f"(Yani, model {len(y_test)} hastanın {int(accuracy * len(y_test))} tanesini doğru tahmin etti.)")

# b) Detaylı Rapor (Precision, Recall)
# Bu, modelimizin nerede hata yaptığını daha detaylı gösterir
print("\n--- Detaylı Sınıflandırma Raporu ---")
print(classification_report(y_test, y_pred))

--- Model Doğruluk (Accuracy) Skoru ---
86.67%
(Yani, model 60 hastanın 52 tanesini doğru tahmin etti.)

--- Detaylı Sınıflandırma Raporu ---
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        36
           1       0.83      0.83      0.83        24

    accuracy                           0.87        60
   macro avg       0.86      0.86      0.86        60
weighted avg       0.87      0.87      0.87        60



In [ ]:
# Gerekli tüm kütüphanelerimizi bir arada tutalım
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # <-- YENİ KÜTÜPHANE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print("✓ Gerekli kütüphaneler (StandardScaler dahil) yüklendi.")

✓ Gerekli kütüphaneler (StandardScaler dahil) yüklendi.


In [ ]:
# Adım 3.2'nin aynısı (X ve y'nin zaten var olduğunu varsayıyoruz)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim seti boyutu: {len(X_train)}")
print(f"Test seti boyutu: {len(X_test)}")

Eğitim seti boyutu: 237
Test seti boyutu: 60


In [ ]:
# 1. Ölçeklendiriciyi (Scaler) oluştur
scaler = StandardScaler()

# 2. ÖLÇEĞİ ÖĞREN ve EĞİTİM VERİSİNİ DÖNÜŞTÜR
# .fit_transform() -> SADECE X_train üzerinde kullanılır
# Bu, 'datayı sızdırmamak' (data leakage) için çok önemlidir
X_train_scaled = scaler.fit_transform(X_train)

# 3. TEST VERİSİNİ DÖNÜŞTÜR
# .transform() -> Test verisine, eğitim verisinden öğrendiğimiz
# ölçeği (ortalama ve std) uygularız.
X_test_scaled = scaler.transform(X_test)

print("✓ Eğitim ve Test verileri başarıyla ölçeklendirildi.")
print("Orijinal X_train ilk satır, 'age' değeri:", X_train.iloc[0, 0])
print("Ölçeklenmiş X_train_scaled ilk satır, 'age' değeri:", X_train_scaled[0, 0])

✓ Eğitim ve Test verileri başarıyla ölçeklendirildi.
Orijinal X_train ilk satır, 'age' değeri: 39
Ölçeklenmiş X_train_scaled ilk satır, 'age' değeri: -1.7416785320687815


In [ ]:
# 1. Modeli oluştur (Aynı model)
model_scaled = LogisticRegression(max_iter=1000)

# 2. Modeli EĞİT (Fit) -> Ölçeklenmiş VERİ ile
model_scaled.fit(X_train_scaled, y_train)

print("✓ Ölçeklenmiş veri ile model başarıyla eğitildi.")
print("(Not: Artık 'ConvergenceWarning' uyarısı almamalısınız!)")

# 3. TAHMİN Yap (Predict) -> Ölçeklenmiş TEST VERİSİ ile
y_pred_scaled = model_scaled.predict(X_test_scaled)

# 4. DEĞERLENDİR (Evaluate)
accuracy_scaled = accuracy_score(y_test, y_pred_scaled)

print(f"\n--- YENİ Model Doğruluk (Accuracy) Skoru (Ölçeklenmiş) ---")
print(f"{accuracy_scaled * 100:.2f}%")

print("\n--- YENİ Detaylı Sınıflandırma Raporu (Ölçeklenmiş) ---")
print(classification_report(y_test, y_pred_scaled))

✓ Ölçeklenmiş veri ile model başarıyla eğitildi.
(Not: Artık 'ConvergenceWarning' uyarısı almamalısınız!)

--- YENİ Model Doğruluk (Accuracy) Skoru (Ölçeklenmiş) ---
88.33%

--- YENİ Detaylı Sınıflandırma Raporu (Ölçeklenmiş) ---
              precision    recall  f1-score   support

           0       0.91      0.89      0.90        36
           1       0.84      0.88      0.86        24

    accuracy                           0.88        60
   macro avg       0.88      0.88      0.88        60
weighted avg       0.88      0.88      0.88        60



In [ ]:
# --- Adım 5: Daha Güçlü Bir Model Deneyelim (Random Forest) ---

# 1. Gerekli modeli 'import' edelim
from sklearn.ensemble import RandomForestClassifier

# 2. Modeli oluştur
# random_state=42 -> Tekrarlanabilir sonuçlar için
model_rf = RandomForestClassifier(random_state=42)

# 3. Modeli EĞİT (Fit) -> Ölçeklenmiş VERİ ile
model_rf.fit(X_train_scaled, y_train)

print("✓ Random Forest modeli başarıyla eğitildi.")

# 4. TAHMİN Yap (Predict)
y_pred_rf = model_rf.predict(X_test_scaled)

# 5. DEĞERLENDİR (Evaluate)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"\n--- Random Forest Model Doğruluk (Accuracy) Skoru ---")
print(f"{accuracy_rf * 100:.2f}%")

print("\n--- Random Forest Detaylı Sınıflandırma Raporu ---")
print(classification_report(y_test, y_pred_rf))

✓ Random Forest modeli başarıyla eğitildi.

--- Random Forest Model Doğruluk (Accuracy) Skoru ---
88.33%

--- Random Forest Detaylı Sınıflandırma Raporu ---
              precision    recall  f1-score   support

           0       0.91      0.89      0.90        36
           1       0.84      0.88      0.86        24

    accuracy                           0.88        60
   macro avg       0.88      0.88      0.88        60
weighted avg       0.88      0.88      0.88        60

